In [1]:
# Student name: Jiayan Dong
# PID: A16593051
# Date: 11/3/2022
# [HW5_prob1] Resnet20 Quantization-aware Training (15pts)
# 2 bit

import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "resnet20_quant_2bit"
model = resnet20_quant(bit=2)

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar(
  (conv1): QuantConv2d(
    bit=2,
    3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (weight_quant): weight_quantize_fn()
  )
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        bit=2,
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        bit=2,
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        bit=2,
        16, 

In [ ]:
# This cell won't be given, but students will complete the training

lr = 0.12
weight_decay = 1e-4
epochs = 200
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [2]:
# HW 5 Q1 2 bit

#  1. Train with 2 bits for both weight and activation to achieve >90% accuracy
#  I can only up to 67.66% accuracy for 2 bit Quantization-aware Training

In [3]:
PATH = "result/resnet20_quant_2bit/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 6766/10000 (68%)



In [4]:
#send an input and grap the value by using prehook like HW3
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu") 

class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, QuantConv2d):
        # print("prehooked")
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)
# From discussion question: Question on pre-hooked inputs for Resnet model by Rishav Karki
print("pre-hooked input of BasicBlock 0 conv1 size:", save_output.outputs[0][0].size())
print("pre-hooked input of BasicBlock 0 conv2 size:", save_output.outputs[1][0].size())

pre-hooked input of BasicBlock 0 conv1 size: torch.Size([128, 3, 32, 32])
pre-hooked input of BasicBlock 0 conv2 size: torch.Size([128, 16, 32, 32])


In [5]:
w_bit = 2
weight_q = model.layer1[0].conv1.weight_q # quantized value is stored during the training
w_alpha = model.layer1[0].conv1.weight_quant.wgt_alpha.data.item()
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[ 0., -0., -0.],
          [ 0., -0., -0.],
          [ 0.,  1.,  1.]],

         [[ 0., -0., -0.],
          [ 0., -0., -0.],
          [ 0.,  0., -0.]],

         [[ 0.,  0.,  1.],
          [ 0., -1., -1.],
          [ 1., -0., -0.]],

         ...,

         [[-0., -0.,  0.],
          [-0.,  0.,  0.],
          [-0., -0., -0.]],

         [[-0., -1., -0.],
          [-0., -0., -0.],
          [ 0.,  1.,  0.]],

         [[ 0., -0., -0.],
          [ 0.,  0., -0.],
          [ 0.,  0.,  0.]]],


        [[[-0., -0., -0.],
          [-0., -0., -0.],
          [-1., -0., -0.]],

         [[-0.,  0.,  0.],
          [-0., -0.,  0.],
          [-0.,  0., -0.]],

         [[ 0.,  0., -0.],
          [ 0.,  0., -0.],
          [ 0.,  0.,  0.]],

         ...,

         [[-0.,  0.,  0.],
          [ 0.,  0.,  0.],
          [-0., -0., -0.]],

         [[-0., -0., -1.],
          [-0., -0., -0.],
          [-1.,  0., -0.]],

         [[-0.,  0.,  0.],
          [-0., -0.,  0.],
 

In [6]:
x_bit = 2
x = save_output.outputs[1][0]  # input of the 2nd conv layer
x_alpha  = model.layer1[0].conv1.act_alpha.data.item()
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 2., 2.],
          [1., 1., 2.,  ..., 2., 2., 2.],
          ...,
          [2., 3., 3.,  ..., 3., 3., 2.],
          [2., 3., 3.,  ..., 3., 3., 2.],
          [2., 3., 3.,  ..., 3., 3., 2.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[1., 1., 1.,  ..., 1., 1., 0.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [2., 1., 1.,  ..., 1., 1., 1.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 

In [7]:
conv_int = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int = conv_int(x_int)
output_recovered = output_int*w_delta*x_delta
print(output_recovered)

tensor([[[[ 2.3087e+01,  3.0783e+01,  2.3087e+01,  ...,  1.5392e+01,
            1.5392e+01,  7.6958e+00],
          [ 3.0783e+01,  7.6958e+00, -7.6958e+00,  ...,  7.6958e+00,
            0.0000e+00, -1.5392e+01],
          [-1.5392e+01, -1.5392e+01, -1.5392e+01,  ...,  2.3087e+01,
            2.3087e+01,  7.6958e+00],
          ...,
          [ 1.1544e+02,  1.1544e+02,  1.1544e+02,  ...,  7.6958e+01,
            8.4654e+01,  6.1567e+01],
          [ 5.3871e+01,  5.3871e+01,  5.3871e+01,  ...,  3.0783e+01,
            3.8479e+01,  2.3087e+01],
          [ 5.3871e+01,  4.6175e+01,  4.6175e+01,  ...,  4.6175e+01,
            4.6175e+01,  3.0783e+01]],

         [[-3.8479e+01, -3.8479e+01, -4.6175e+01,  ..., -3.8479e+01,
           -3.8479e+01, -4.6175e+01],
          [-3.8479e+01, -4.6175e+01, -3.0783e+01,  ..., -3.8479e+01,
           -3.8479e+01, -3.8479e+01],
          [-3.8479e+01, -3.8479e+01, -3.0783e+01,  ..., -3.8479e+01,
           -3.8479e+01, -3.8479e+01],
          ...,
     

In [8]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
conv_ref.weight = model.layer1[0].conv1.weight_q

output_ref = conv_ref(x)
print(output_ref)

tensor([[[[ 3.1603e+01,  2.3689e+01,  2.3098e+01,  ...,  1.0395e+01,
            1.7835e+01,  1.4673e+01],
          [ 2.0346e+01,  9.4218e+00, -8.3211e+00,  ...,  6.3169e+00,
            9.5273e+00, -6.8256e+00],
          [-2.2703e+01, -1.3768e+01, -2.1790e+01,  ...,  2.5034e+01,
            2.4249e+01,  1.3520e+01],
          ...,
          [ 1.1342e+02,  1.1448e+02,  1.1640e+02,  ...,  7.3226e+01,
            6.8186e+01,  5.6231e+01],
          [ 4.9716e+01,  4.4870e+01,  4.5654e+01,  ...,  3.8330e+01,
            3.8423e+01,  3.1601e+01],
          [ 4.3970e+01,  4.0478e+01,  4.0478e+01,  ...,  4.0478e+01,
            4.0478e+01,  3.1464e+01]],

         [[-3.3666e+01, -4.1714e+01, -4.2076e+01,  ..., -2.9825e+01,
           -3.6796e+01, -4.3564e+01],
          [-3.3249e+01, -4.1452e+01, -3.9461e+01,  ..., -3.4542e+01,
           -3.9564e+01, -4.2332e+01],
          [-2.9102e+01, -3.4437e+01, -3.4597e+01,  ..., -3.8792e+01,
           -4.1199e+01, -4.2479e+01],
          ...,
     

In [9]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
weight = model.layer1[0].conv1.weight
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))

output_ref = conv_ref(x)
print(output_ref)


tensor([[[[ 5.8381e+01,  5.0363e+01,  5.2783e+01,  ...,  3.7802e+01,
            4.1512e+01,  4.0391e+01],
          [ 4.1823e+01,  3.1472e+01,  1.8835e+01,  ...,  3.3390e+01,
            3.4245e+01,  1.6219e+01],
          [ 1.9531e+00,  1.2765e+01,  1.2624e+01,  ...,  4.8955e+01,
            4.3899e+01,  2.9857e+01],
          ...,
          [ 1.2508e+02,  1.2485e+02,  1.2970e+02,  ...,  6.5126e+01,
            5.6717e+01,  4.6700e+01],
          [ 3.5530e+01,  2.7888e+01,  2.8718e+01,  ...,  1.3559e+01,
            1.1585e+01,  8.6948e+00],
          [ 1.2413e+01,  4.2271e+00,  4.2271e+00,  ...,  4.2271e+00,
            4.2271e+00,  4.3258e+00]],

         [[-7.1085e+01, -8.0673e+01, -7.7732e+01,  ..., -6.1991e+01,
           -7.3917e+01, -8.4541e+01],
          [-6.8568e+01, -7.6548e+01, -7.3948e+01,  ..., -6.7028e+01,
           -7.7548e+01, -8.4694e+01],
          [-6.0522e+01, -6.2529e+01, -6.2380e+01,  ..., -7.7196e+01,
           -8.4078e+01, -8.5889e+01],
          ...,
     

In [10]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(39.7482, device='cuda:0', grad_fn=<MeanBackward0>)
